### Daten importieren

In [3]:
import pandas as pd
import plotly.express as px
import numpy as np

### Daten auslesen

In [21]:
df = pd.read_csv('flower_data.csv')

dict = {'name': 'Name',
        'height (cm)': 'Höhe',
        'longevity (years)': 'Lebenserwartung',
        'place of origin': 'Herkunftskontinent',
        'color': 'Farbe',
        'climate': 'Klima',
        'cut flowers': 'Geschnittene Blume',
        'parfumes': "Parfum",
        'medicine': 'Medizin',
        'average number of petals': 'Durchschnittliche Anzhal Blätter'}

df.rename(columns=dict, inplace=True)

display(df)

,Name,Höhe,Lebenserwartung,Herkunftskontinent,Farbe,Klima,Geschnittene Blume,perfumes,Medizin,Durchschnittliche Anzhal Blätter
0,Rose,60-150,1-2,"Europe, Asia","Red, pink, white",Temperate,1,1,1,5-9
1,Tulip,20-70,2-3,Central Asia,"Red, pink, white",Temperate,1,0,1,3-7
2,Daisy,15-60,1-2,"Europe, Asia","White, yellow",Temperate,1,0,1,10-20
3,Sunflower,50-300,1-2,North and South America,"Yellow, orange",Temperate to tropical,1,0,1,10-20
4,Orchid,20-100,5-10,Worldwide,"Purple, pink, white",Tropical,1,1,1,3-50
...,...,...,...,...,...,...,...,...,...,...
67,Hyacinth,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10
68,Iris reticulata,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10
69,Chionodoxa,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10
70,Anemone,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10


### Daten filtern

In [23]:
# Split the "Herkunftskontinent" column
df["Herkunftskontinent"] = df["Herkunftskontinent"].str.split(", ")

# Explode the DataFrame
df = df.explode("Herkunftskontinent")

# Filter the DataFrame for "Asia" and "Europe"
df_filtered = df.loc[df["Herkunftskontinent"].isin(["Asia", "Europe"])]

# Print the resulting DataFrame
df_filtered.head()

,Name,Höhe,Lebenserwartung,Herkunftskontinent,Farbe,Klima,Geschnittene Blume,perfumes,Medizin,Durchschnittliche Anzhal Blätter
0,Rose,60-150,1-2,Europe,"Red, pink, white",Temperate,1,1,1,5-9
0,Rose,60-150,1-2,Asia,"Red, pink, white",Temperate,1,1,1,5-9
2,Daisy,15-60,1-2,Europe,"White, yellow",Temperate,1,0,1,10-20
2,Daisy,15-60,1-2,Asia,"White, yellow",Temperate,1,0,1,10-20
5,Jasmine,60-180,5-10,Asia,"White, yellow",Tropical,1,1,1,5-9


### Kuchen-Diagramm: Herkunftskontinent

In [38]:
font_family = 'Arial'

df["Herkunftskontinent"] = df["Herkunftskontinent"].str.split(", ")
df = df.explode("Herkunftskontinent")

origin_counts = df["Herkunftskontinent"].value_counts()

colors = px.colors.qualitative.Pastel

fig = px.pie(values=origin_counts, names=origin_counts.index, title="Herkunftskontinent", hole=0.5,
             color_discrete_sequence=colors)

fig.update_layout(title={'font': {'family': font_family, 'size': 24}, 'x': 0.5, 'y': 0.9},
                  font={'family': font_family, 'size': 12})

fig.show()

TypeError: 'dict' object is not callable

### Kuchen-Diagramm: Europa vs Asien

In [35]:
font_family = 'Arial'

asia_europe = df_filtered["Herkunftskontinent"].value_counts()

colors = px.colors.qualitative.Pastel

fig = px.pie(values=asia_europe, names=asia_europe.index, title="Asia vs Europe", hole=0.5)

fig.update_layout(title={'font': {'family': font_family, 'size': 24}, 'x': 0.5, 'y': 0.9},
                  font={'family': font_family, 'size': 12})

fig.show()